In [2]:
import sys
# sys.path.append('../../')

from os import listdir

from hdf5storage import loadmat, savemat 
import numpy as np 
from matplotlib import pyplot as plt

from scipy import signal 
# from hnlpy.timeop import makefiltersos
# from hnlpy.timeop import narrowfilter 
from scipy.signal import sosfiltfilt
from scipy.signal import hilbert
from scipy.signal import savgol_filter
from scipy.stats import sem
from numpy.polynomial.polynomial import polyfit, polyval

import networkx as nx

In [3]:
# organize file in time sequence
filedates=[20220713,20220721,20220804,20220808,20220810,20220811,20220815,20220816,20221003,2022100401,2022100402,20221005]
numSes=len(filedates)

In [4]:
ses=0
outdict = loadmat('../Filter_Hilbert/Hurst_eeg_20_'+str(filedates[ses])+'.mat')
outdict.keys()

dict_keys(['band_labels', 'channel_labels', 'channels', 'condition_names', 'eeg_Hurst', 'filename', 'intervals', 'samples', 'session', 'sessionTypes'])

In [ ]:
outdict = loadmat('../Filter_Hilbert/Hurst_eeg_20_'+str(filedates[0])+'.mat')
outdict.keys()

In [ ]:
# put everything in 144 first dimenstion 
eeg_Hurst144x2       =np.zeros((144,2,7,32))
condition_index144   =np.zeros((144))
intervals144         =dict()
samples144           =dict()
session144           =np.zeros((144))
n=0
for ses in range(12):
    outdict = loadmat('../Filter_Hilbert/Hurst_eeg_20_'+str(filedates[ses])+'.mat')
    eeg_Hurst           = outdict['eeg_Hurst']          # 12x2x7x32
    condition_index     = outdict['condition_index']    # 12
    intervals           = outdict['intervals']          # 12x175x2
    samples             = outdict['samples']            # 12x175x2
    session             = outdict['session']            # 12
    if ses == 0: # propagate the common variables and save in the mat file
        new_sr              = outdict['new_sr']             
        passbands           = outdict['passbands']          
        stopbands           = outdict['stopbands']          
        band_labels         = outdict['band_labels']   
        condition_names     = outdict['condition_names']    
        channels            = outdict['channels']           
        channel_labels      = outdict['channel_labels']  
        sessionTypes        = outdict['sessionTypes']    
    for trl in range(12):
        condition_index144[n]     = condition_index[trl]    # 12
        intervals144[str(n)]      = intervals[trl]
        samples144[str(n)]        = samples[trl]
        session144[n]             = session[trl]
        for subj in range(2):
            eeg_Hurst144x2[n,subj]       = eeg_Hurst[trl,subj,:,:]
        n+=1

outdict = dict()
outdict['eeg_Hurst144x2']       = eeg_Hurst144x2
outdict['condition_index144']   = condition_index144
outdict['intervals144']         = intervals144
outdict['samples144']           = samples144
outdict['session144']           = session144

outdict['new_sr']               = new_sr
outdict['passbands']            = passbands
outdict['stopbands']            = stopbands
outdict['band_labels']          = band_labels
outdict['condition_names']      = condition_names
outdict['channels']             = channels
outdict['channel_labels']       = channel_labels
outdict['sessionTypes']         = sessionTypes

savemat('Hurst_eeg144',outdict,store_python_metadata = True) 

# takes 18 s


In [ ]:
outdict=loadmat('Hurst_eeg144.mat')
eeg_Hurst144x2      = outdict['eeg_Hurst144x2']       
condition_index144  = outdict['condition_index144']   
intervals144        = outdict['intervals144']         
samples144          = outdict['samples144']           
session144          = outdict['session144']

new_sr              = outdict['new_sr']             
passbands           = outdict['passbands']          
stopbands           = outdict['stopbands']          
band_labels         = outdict['band_labels']   
condition_names     = outdict['condition_names']    
channels            = outdict['channels']           
channel_labels      = outdict['channel_labels']  
sessionTypes        = outdict['sessionTypes']  

In [ ]:
# organize indicies
synch_ind=np.where(session144==1)
synco_ind=np.where(session144==2)

Uncoupled_ind=np.where(condition_index144==0)
L_lead_ind   =np.where(condition_index144==1)
R_lead_ind   =np.where(condition_index144==2)
Mutual_ind   =np.where(condition_index144==3)

# Uncoupled condition in Synch for L&R
Uncoupled_synch_ind=np.intersect1d(synch_ind,Uncoupled_ind)
# Uncoupled condition in Synco for L&R
Uncoupled_synco_ind=np.intersect1d(synco_ind,Uncoupled_ind)

# L_Leading in Synch / R_following in Synch
L_lead_synch_ind=np.intersect1d(synch_ind,L_lead_ind)
# R_leading in Synch / L_following in Synch
R_lead_synch_ind=np.intersect1d(synch_ind,R_lead_ind)

# L_Leading in Synco / R_following in Synco
L_lead_synco_ind=np.intersect1d(synco_ind,L_lead_ind)
# R_leading in Synco / L_following in Synco
R_lead_synco_ind=np.intersect1d(synco_ind,R_lead_ind)

# Mutual condition in Synch for L&R
Mutual_synch_ind=np.intersect1d(synch_ind,Mutual_ind)
# Mutual condition in Synco for L&R
Mutual_synco_ind=np.intersect1d(synco_ind,Mutual_ind)


In [ ]:

# parameters for plotting
states4names=['Uncoupled','Leading','Following','Mutual'] 
syn2names=['Synch','Synco']
states3names=['Independent','Unidirectional','Bidirectional']

color_4st=['green','red','blue','black']